### 최신 정보를 알려주지 못하는 GPT

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-4o-mini")
model.invoke("최슨 로제가 발표한 신곡은 무엇인가요?")

AIMessage(content='최슨 로제의 최신 곡에 대한 정보는 2023년 10월까지의 데이터에는 포함되어 있지 않습니다. 그녀의 최신 음악이나 발표를 확인하려면 공식 팬 사이트나 음악 스트리밍 플랫폼을 참고해 보시는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 20, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBI175EJDSKZpqCeSDxjHDGtW305K', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8bd59461-47a2-4858-9221-46e786fb6252-0', usage_metadata={'input_tokens': 20, 'output_tokens': 57, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 덕덕고 검색 설치하기

In [5]:
%pip install -U ddgs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 덕덕고 검색 테스트하기

In [25]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(results_separator='\n')
docs = search.invoke("최슨 로제가 발표한 신곡은 무엇인가요?")

print(docs)

snippet: 29일 빌보드가 발표한 최신 차트에 따르면, ‘APT.’는 핫 100 차트에 8위로 진입했습니다. ‘APT.’는 경쾌한 멜로디와 중독성 있는 후렴구로 리스너를 사로잡았죠., title: 로제가 ‘APT.’로 새로 쓴 역사 | 보그 코리아 (Vogue Korea), link: https://www.vogue.co.kr/2024/10/29/로제가-apt-로-새로-쓴-역사/
snippet: 걸그룹 블랙핑크의 로제가 빌보드 글로벌 차트 정상을 찍은 곡 ‘아파트’(APT.) 탄생 비화를 소개했다. 브루노 마스 쪽에 3곡의 협업곡 후보를 제시했는데, 브루노 마스가 ‘아파트’를 콕 집었다는 것이다., title: 로제가 보낸 3곡 중 ‘아파트’ 고른 브루노 마스…“끝내주네” | 한겨레, link: https://www.hani.co.kr/arti/international/international_general/1166261.html
snippet: 로제가 전 세계에 알린 한국 술 게임 아파트, 어떻게 하는 건데? 복사. 공유., title: 로제가 전 세계에 알린 한국 술 게임 APT... | 얼루어 코리아 (Allure Korea), link: https://www.allurekorea.com/2024/10/20/로제가-전-세계에-알린-한국-술-게임-아파트-어떻게-하/
snippet: 6일 로 첫 솔로 앨범을 발표한 블랙핑크 로제. 가창력 넘치는 노래와 날렵한 댄스로 돌아왔어요., title: ‘먹챙’ 로제가 살 안 찌는 이유 - DAILY NEWS, link: https://www.elle.co.kr/article/53680


### 질의 확장 구현하기

In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"아래 context에 기반하여 사용자의 질문에 답변하라.:\n\n{docs}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = question_answering_prompt | model

### 메시지 저장하고 답변 생성 후 출력하기

In [27]:
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory()
# 사용자 질문을 메모리에 저장
chat_history.add_user_message("요즘 로제가 발표한 신곡은 무엇인가요?")

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer)

print(answer)

content="로제가 최근 발표한 신곡은 'APT.'입니다. 이 곡은 핫 100 차트에서 8위로 진입하였고, 경쾌한 멜로디와 중독성 있는 후렴구로 많은 사랑을 받고 있습니다." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 466, 'total_tokens': 524, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d77b305a29', 'id': 'chatcmpl-CBFXEWVxhsZPdLPzaqzyossRn2HPX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--15602275-feba-45e0-a6ba-6e6d1b58c713-0' usage_metadata={'input_tokens': 466, 'output_tokens': 58, 'total_tokens': 524, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
